In [1]:
import numpy as np

In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [3]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle

In [4]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [9]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [26]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder(sparse_output=False)
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder
# geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [27]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [34]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [35]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [31]:
data=pd.concat([data.drop(['Geography'],axis=1),geo_encoder_df],axis=1)

In [32]:
data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [36]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
    
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [37]:
X=data.drop(['Exited'],axis=1)
y=data['Exited']

In [38]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [39]:
y

0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [42]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [44]:
X_train.shape

(8000, 12)

In [45]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [48]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [69]:
models=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [70]:
models.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [71]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
opt=Adam(learning_rate=0.1)

In [64]:
opt

In [72]:
models.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [88]:
log_dir="logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [89]:
early_stopping=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [90]:
history=models.fit(X_train,y_train,validation_data=(X_test,y_test),callbacks=[tensorflow_callback,early_stopping],epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step - accuracy: 0.8665 - loss: 0.3178 - val_accuracy: 0.8570 - val_loss: 0.3453
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step - accuracy: 0.8679 - loss: 0.3165 - val_accuracy: 0.8615 - val_loss: 0.3432
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.8690 - loss: 0.3159 - val_accuracy: 0.8610 - val_loss: 0.3418
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.8699 - loss: 0.3152 - val_accuracy: 0.8630 - val_loss: 0.3408
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - accuracy: 0.8691 - loss: 0.3137 - val_accuracy: 0.8625 - val_loss: 0.3424
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - accuracy: 0.8711 - loss: 0.3120 - val_accuracy: 0.8620 - val_loss: 0.3432
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 551us/step - accuracy: 0.8706 - loss: 0.3131 - val_accuracy: 0.8590 - val_loss: 0.3417
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - accuracy: 0.8704 - loss: 0

In [91]:
models.save('ann_model.h5')

In [92]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [94]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6008 (pid 52376), started 0:00:41 ago. (Use '!kill 52376' to kill it.)

In [95]:
from tensorflow.keras.models import load_model


In [96]:
model=load_model('ann_model.h5')

In [101]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender_pkl=pickle.load(file)

In [102]:
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo_pkl=pickle.load(file)

In [103]:
with open('scaler.pkl','rb') as file:
    scaler_pkl=pickle.load(file)

In [122]:
input_data = {
    'CreditScore': 700,
    'Geography': 'Spain',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 50000.0,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 100000.0
}


In [123]:
geo_encoded=onehot_encoder_geo_pkl.transform([[input_data['Geography']]])
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [124]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,Spain,Male,40,3,50000.0,2,1,1,100000.0


In [126]:
input_df['Gender']=label_encoder_gender_pkl.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,Spain,1,40,3,50000.0,2,1,1,100000.0


In [130]:
input_df=pd.concat([input_df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [131]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,1,40,3,50000.0,2,1,1,100000.0,0.0,0.0,1.0


In [132]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 0.50178794,  0.91324755,  0.10479359, -0.69539349, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.00749869, -0.99850112,
        -0.57946723,  1.73494238]])

In [133]:
prediction=model.predict(input_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [135]:
prediction_prob=prediction[0][0]
prediction_prob


np.float32(0.023197876)

In [136]:
if prediction_prob>0.5:
    print("customer is likely to churn")
else:
    print("customer is not likely to churn")

customer is not likely to churn
